In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pymongo

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw
from climatools.cliradlw.utils import mapband_new2old

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *

import IPython.display as display

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def make_query(param=None):
    return {'param.' + name: value for name, value in param.items()}   

In [4]:
def show_html(s): display.display(display.HTML(s))
    
def show_markdown(s): display.display(display.Markdown(s))

In [5]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [6]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)

    for dim in ['band', 'g']: 
        if dim in ds: ds = ds.sum(dim)

    for dim in ['igg', 'g', 'i']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
                
#    if 'band' in ds.dims:
#        try:
#            ds = ds.squeeze('band')
#        except ValueError:
#            ds = ds.sum('band')     
    return ds['coolrg']

def pltdata_cooling_1():
    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(DS_COOL.sel(i=1)),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(DS_COOL_CRD),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(DS_COOL_WGT.sel(igg=10)),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data

def nice_xlims(pltdata=None, prange=None):
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)

def plt_cool(pltdata=None):
    plin = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='linear',
                                  prange=(50, 1050))
    plin.plot_width, plin.plot_height = 400, 500
    plog = plt_vert_profile_bokeh(pltdata=pltdata,
                                  y_axis_type='log',
                                  prange=(.01, 200))
    plog.plot_width, plog.plot_height = 400, 500
    everything = gridplot(plin, plog, ncols=2)
    return everything
    
def script_plt_cooling():
    pltdata = pltdata_cooling_1()
    p = plt_cool(pltdata=pltdata)
    show(p)
    show_markdown('''*FIGURE.* Cooling rate profiles.''')

In [7]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels: ds = ds.isel(pressure=ilevels)

    for dim in ['band', 'g']:
        if dim in ds: ds = ds.sum(dim)
        
    for dim in ['i', 'igg', 'g']:
        if dim in ds:
            if ds[dim].shape == (1,): ds = ds.squeeze(dim).drop(dim)
            elif ds[dim].shape == (): ds = ds.drop(dim)
    
#    if 'band' in ds.dims:
#        try:
#            ds = ds.squeeze('band').drop('band')
#        except ValueError:
#            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df

def tbdata_flux():
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    atmpro = PARAM['atmpro']
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(DS_FLUX_CRD, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10',
            'df': fmt_flux(DS_FLUX_WGT.sel(igg=10), ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(DS_FLUX.sel(i=1), ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data

def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        show_markdown(f'''*TABLE.*  Fluxes. {d['label']}''')
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        show_markdown(
            f'''*TABLE.*  Fluxes.  ({d['label']}) - ({d0['label']})''')
        
    show_tb(benchmark)
    for d in others: show_tb(d)
    for d in others: show_tbdiff(d=d, d0=benchmark)
        
def script_tb_flux():
    data = tbdata_flux()
    tb_flux(tbdata=data)

In [8]:
def gasbands():
    bmap = mapband_new2old()
    gases = [gas for gas, _ in PARAM['molecule'].items()] 
    bands = [bmap[b] for b in PARAM['band']]
    return list(itertools.product(gases, bands))

def print_bestfit_params():
    df = pd.DataFrame()
    for gas, band in gasbands():
        param = bestfits.kdist_params(molecule=gas, band=band)
        srs = pd.Series(param)
        df[f'({gas}, {band})'] = srs
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df.sort_index())
    pd.reset_option('display.max_colwidth')
        
def print_input_param():
    df_clirad = pd.Series(PARAM, name='clirad run').to_frame()
    df_lblnew = pd.Series(PARAM_LBLNEW, name='lblnew run compared against').to_frame()
    df = pd.merge(df_clirad, df_lblnew, how='outer', 
                  left_index=True, right_index=True)
    df.fillna('-', inplace=True)
    pd.set_option('display.max_colwidth', -1)
    display.display(df)
    pd.reset_option('display.max_colwidth')
        
def print_lblnew_param():
    pd.set_option('display.max_colwidth', -1)
    display.display(pd.Series(PARAM_LBLNEW).to_frame())
    pd.reset_option('display.max_colwidth')

In [9]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL.sel(i=1)]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [10]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX.sel(i=1)]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))

In [11]:
def script_fluxcoolr_excelfile():
    '''
    Save fluxes and cooling rates by layer by g-interval in excel file
    and make it available for download.
    '''
    fname = 'output_byg.xlsx'
    
    writer = pd.ExcelWriter(fname)
    
    for g in DS_FLUX_WGT.coords['g']:
        df = DS_FLUX_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'flux g={int(g)}')
        
    for g in DS_COOL_WGT.coords['g']:
        df = DS_COOL_WGT.sel(igg=1).sel(g=g).to_dataframe()
        df.to_excel(writer, f'cool g={int(g)}')    

In [12]:


    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()

In [13]:
%run param.py

client = pymongo.MongoClient('localhost', 27017)

if 'ng_refs' in PARAM_LBLNEW: 
    doc = client.lblnew.bestfit_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_fluxg']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolrg']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wfluxg']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolrg']))
else:
    doc = client.lblnew.overlap_lw.find_one(make_query(param=PARAM_LBLNEW))
    DS_FLUX_CRD = load_output_file(io.StringIO(doc['output_flux']))
    DS_COOL_CRD = load_output_file(io.StringIO(doc['output_coolr']))
    DS_FLUX_WGT = load_output_file(io.StringIO(doc['output_wflux']))
    DS_COOL_WGT = load_output_file(io.StringIO(doc['output_wcoolr']))

doc = client.cliradnew.lw.find_one(make_query(param=PARAM))
DS_FLUX = load_output_file(io.StringIO(doc['output_flux']))
DS_COOL = load_output_file(io.StringIO(doc['output_coolr']))

In [14]:
script()
if 'ng_refs' in PARAM_LBLNEW:
    script_fluxcoolr_excelfile()

### CLIRAD-LW

### Table of Contents

# Input Parameters

,clirad run,lblnew run compared against
atmpro,trp,trp
band,[3],3a
commitnumber,1013f91,5014a19
dv,-,0.001
molecule,"{'co2': 0.0004, 'h2o': 'atmpro', 'n2o': 3.2e-07}","{'h2o': 'atmpro', 'co2': 0.0004, 'n2o': 3.2e-07}"
nv,-,1000
tsfc,300,300


# Best-fit Parameters

...

*TABLE.*  Fluxes.  (CLIRAD) - (CRD)

# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.393544                    1  0.251587
0.000750        2  0.393826                    2  0.256587
0.001052        3  0.407652                    3  0.269264
0.001476        4  0.419989                    4  0.281879
0.002070        5  0.430245                    5  0.294185
0.002904        6  0.437232                    6  0.305671
0.004074        7  0.440234                    7  0.316105
0.005714        8  0.437915                    8  0.324703
0.008015        9  0.428933                    9  0.330530
0.011243       10  0.411771                   10  0.332535
0.015771       11  0.384794                   11  0.328838
0.022122       12  0.346205                   12  0.317106
0.031031       13  0.293922                   13  0.294062
0.043528       14  0.229687                   14  0.260269
0.061057       15  0.208438                   15  0.258687
0.085645       16  0.257563                   16  0.310882
0.120136       17  0.321923                   17  0.367102
0.168516       18  0.371046                   18  0.391524
0.236378       19  0.414875                   19  0.392014
0.331549       20  0.469674                   20  0.402987
0.465100       21  0.560351                   21  0.472663
0.652400       22  0.699836                   22  0.618243
0.915100       23  0.704428                   23  0.658386
1.283650       24  0.598948                   24  0.586342
1.800600       25  0.507972                   25  0.505027
2.525700       26  0.412678                   26  0.414910
3.542800       27  0.336798                   27  0.351002
4.969550       28  0.276840                   28  0.303774
6.970850       29  0.226215                   29  0.260656
9.778100       30  0.182400                   30  0.211640
13.715850      31  0.144233                   31  0.161741
19.239350      32  0.111211                   32  0.116377
26.987250      33  0.083351                   33  0.079985
37.855300      34  0.061138                   34  0.056054
53.100050      35  0.030482                   35  0.029744
73.887500      36 -0.005806                   36 -0.003713
97.662500      37 -0.028726                   37 -0.027252
121.437500     38 -0.025035                   38 -0.023354
145.212500     39 -0.004890                   39 -0.002700
168.987500     40  0.009253                   40  0.011680
192.762500     41  0.023001                   41  0.025636
216.537500     42  0.039931                   42  0.041438
240.312500     43  0.060557                   43  0.059988
264.087500     44  0.084200                   44  0.080988
287.862500     45  0.109394                   45  0.102715
311.637500     46  0.135228                   46  0.125325
335.412500     47  0.161308                   47  0.150133
359.187500     48  0.186203                   48  0.172865
382.962500     49  0.204963                   49  0.186648
406.737500     50  0.225413                   50  0.203070
430.512500     51  0.246860                   51  0.224737
454.287500     52  0.268816                   52  0.248806
478.062500     53  0.290744                   53  0.271773
501.837500     54  0.311660                   54  0.291545
525.612500     55  0.331270                   55  0.310888
549.387500     56  0.348548                   56  0.329413
573.162500     57  0.363046                   57  0.347597
596.937500     58  0.374588                   58  0.363750
620.712500     59  0.382803                   59  0.376223
644.487500     60  0.388297                   60  0.385632
668.262500     61  0.390868                   61  0.392076
692.037500     62  0.388423                   62  0.394040
715.812500     63  0.377532                   63  0.388652
739.587500     64  0.372229                   64  0.391360
763.362500     65  0.345107                   65  0.371915
78

# Fluxes by Layer

CRD                           CLIRAD                \
                        flug       fldg      fnetg       flug          fldg   
pressure    level                                                             
0.000000    1     -25.153740   0.000000 -25.153740 -25.455290  4.438955e-08   
0.000624    2     -25.153758   0.000048 -25.153710 -25.455308  3.620268e-05   
0.000876    3     -25.153766   0.000067 -25.153699 -25.455315  5.089270e-05   
0.001229    4     -25.153776   0.000095 -25.153682 -25.455325  7.192247e-05   
0.001723    5     -25.153791   0.000134 -25.153657 -25.455338  1.020213e-04   
0.002417    6     -25.153811   0.000189 -25.153622 -25.455357  1.450475e-04   
0.003391    7     -25.153839   0.000267 -25.153571 -25.455383  2.064623e-04   
0.004757    8     -25.153877   0.000377 -25.153500 -25.455420  2.939561e-04   
0.006672    9     -25.153930   0.000529 -25.153401 -25.455470  4.182373e-04   
0.009359    10    -25.154004   0.000740 -25.153264 -25.455541  5.940686e-04   
0.013128    11    -25.154107   0.001027 -25.153080 -25.455640  8.415560e-04   
0.018415    12    -25.154254   0.001414 -25.152839 -25.455780  1.187392e-03   
0.025830    13    -25.154462   0.001927 -25.152535 -25.455979  1.665445e-03   
0.036232    14    -25.154767   0.002594 -25.152173 -25.456268  2.316458e-03   
0.050823    15    -25.155225   0.003449 -25.151776 -25.456692  3.190850e-03   
0.071291    16    -25.155898   0.004627 -25.151271 -25.457300  4.425838e-03   
0.100000    17    -25.156815   0.006420 -25.150395 -25.458112  6.294856e-03   
0.140271    18    -25.157998   0.009139 -25.148859 -25.459135  9.069079e-03   
0.196760    19    -25.159464   0.013088 -25.146376 -25.460367  1.292130e-02   
0.275997    20    -25.161175   0.018693 -25.142482 -25.461751  1.798585e-02   
0.387100    21    -25.162935   0.026634 -25.136301 -25.463094  2.463313e-02   
0.543100    22    -25.164042   0.038096 -25.125946 -25.463768  3.404236e-02   
0.761700    23    -25.162384   0.054560 -25.107824 -25.461885  4.816995e-02   
1.068500    24    -25.157088   0.074865 -25.082223 -25.456658  6.687255e-02   
1.498800    25    -25.149188   0.097495 -25.051693 -25.448908  8.901339e-02   
2.102400    26    -25.138748   0.123375 -25.015373 -25.438598  1.148168e-01   
2.949000    27    -25.126572   0.152584 -24.973987 -25.426332  1.441649e-01   
4.136600    28    -25.112872   0.186266 -24.926607 -25.412091  1.793072e-01   
5.802500    29    -25.097565   0.225589 -24.871976 -25.395634  2.228025e-01   
8.139200    30    -25.080739   0.271379 -24.809359 -25.376762  2.760874e-01   
11.417000   31    -25.062790   0.324253 -24.738538 -25.355808  3.373175e-01   
16.014700   32    -25.044626   0.384642 -24.659984 -25.333698  4.033050e-01   
22.464000   33    -25.027922   0.452900 -24.575022 -25.312497  4.710210e-01   
31.510500   34    -25.015168   0.529467 -24.485701 -25.295271  5.395184e-01   
44.200100   35    -25.007926   0.614125 -24.393801 -25.283661  6.121758e-01   
62.000000   36    -25.023453   0.693925 -24.329529 -25.296441  6.876784e-01   
85.775000   37    -25.101329   0.755448 -24.345880 -25.378518  7.592980e-01   
109.550000  38    -25.236454   0.809672 -24.426782 -25.525063  8.290847e-01   
133.325000  39    -25.410049   0.912762 -24.497287 -25.712708  9.509513e-01   
157.100000  40    -25.598522   1.087464 -24.511058 -25.913773  1.144411e+00   
180.875000  41    -25.798937   1.313939 -24.484998 -26.124536  1.388073e+00   
204.650000  42    -26.009904   1.589684 -24.420219 -26.343893  1.679635e+00   
228.425000  43    -26.233245   1.925483 -24.307762 -26.574309  2.026766e+00   
252.200000  44    -26.469663   2.332448 -24.137215 -26.815981  2.437401e+00   
275.975000  45    -26.720428   2.820348 -23.900080 -27.069707  2.919240e+00   
299.750000  46    -26.987050   3.395059 -23.591991 -27.336785  3.475625e+00   
323.525000  47    -27.271151   4.060004 -23.211147 -27.618594  4.110426e+00   
347.300000  48    -27.574500   4.817648 -22.756853 -27.916803  4.831500e+00

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')